# Introduction to Neural Networks with TensorFlow and PyTorch

This notebook demonstrates fundamental neural network concepts using both TensorFlow/Keras and PyTorch.

## Sections:
1. Perceptron from Scratch
2. Multi-Layer Perceptron with NumPy
3. Keras Neural Network
4. PyTorch Neural Network
5. Model Evaluation
6. Model Deployment (SavedModel, TorchScript, TFLite, ONNX)

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

print(f'TensorFlow version: {tf.__version__}')
print(f'PyTorch version: {torch.__version__}')

## 1. Perceptron from Scratch

In [ ]:
class Perceptron:
    """Simple perceptron for binary classification"""
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.lr = learning_rate
        self.n_iter = n_iterations
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        for _ in range(self.n_iter):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self._activation(linear_output)
                update = self.lr * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update
    
    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self._activation(linear_output)
    
    def _activation(self, x):
        return np.where(x >= 0, 1, 0)

# Test the perceptron
X_simple, y_simple = make_classification(n_samples=100, n_features=2, n_redundant=0, 
                                         n_informative=2, random_state=42, n_clusters_per_class=1)
perceptron = Perceptron(learning_rate=0.1, n_iterations=100)
perceptron.fit(X_simple, y_simple)
predictions = perceptron.predict(X_simple)
print(f'Perceptron Accuracy: {accuracy_score(y_simple, predictions):.4f}')

## 2. Multi-Layer Perceptron with NumPy

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def sigmoid_derivative(x):
    return x * (1 - x)

class NumpyMLP:
    """Simple MLP with one hidden layer using NumPy"""
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.lr = learning_rate
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y):
        m = X.shape[0]
        dz2 = self.a2 - y.reshape(-1, 1)
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        dz1 = np.dot(dz2, self.W2.T) * sigmoid_derivative(self.a1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1
    
    def train(self, X, y, epochs=1000):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y)
            if epoch % 200 == 0:
                loss = np.mean((output - y.reshape(-1, 1)) ** 2)
                print(f'Epoch {epoch}, Loss: {loss:.4f}')

# Test NumPy MLP
numpy_mlp = NumpyMLP(input_size=2, hidden_size=4, output_size=1, learning_rate=0.5)
numpy_mlp.train(X_simple, y_simple, epochs=1000)
numpy_predictions = (numpy_mlp.forward(X_simple) > 0.5).astype(int).flatten()
print(f'NumPy MLP Accuracy: {accuracy_score(y_simple, numpy_predictions):.4f}')

## 3. Keras Neural Network

### Data Preparation

In [ ]:
# Generate a more complex dataset for Keras and PyTorch
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, 
                          n_redundant=5, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'Training set shape: {X_train_scaled.shape}')
print(f'Test set shape: {X_test_scaled.shape}')

### Build and Train Keras Model

In [ ]:
# Build Keras model
model_keras = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(20,)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model_keras.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

print(model_keras.summary())

# Train the model
history = model_keras.fit(X_train_scaled, y_train,
                         epochs=50,
                         batch_size=32,
                         validation_split=0.2,
                         verbose=0)

print(f'\nFinal training accuracy: {history.history["accuracy"][-1]:.4f}')
print(f'Final validation accuracy: {history.history["val_accuracy"][-1]:.4f}')

### Save Keras Model

In [ ]:
# Save model in SavedModel format
saved_model_dir = 'keras_saved_model'
model_keras.save(saved_model_dir, include_optimizer=False)
print(f'Saved Keras model to {saved_model_dir}')

In [ ]:
# === TFLite conversion and simple inference example ===
# NEW CELL ADDED: TFLite export for edge deployment
import tensorflow as tf
import numpy as np

saved_model_dir = 'keras_saved_model'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# Optional optimizations for edge: default optimize, float16 quant, or full integer quant
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Example: allow float16 quantization (works well on many edge devices)
converter.target_spec.supported_types = [tf.float16]

try:
    tflite_model = converter.convert()
    tflite_path = 'keras_model.tflite'
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    print('Saved TFLite model to', tflite_path)

    # Run a quick inference using the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Create a dummy input with correct shape
    input_shape = input_details[0]['shape']
    dummy_input = np.random.randn(*[dim if dim != -1 else 1 for dim in input_shape]).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], dummy_input)
    interpreter.invoke()
    out = interpreter.get_tensor(output_details[0]['index'])
    print('TFLite inference output (shape):', out.shape)
except Exception as e:
    print('TFLite conversion/inference failed:', e)
    print('If running locally, install or upgrade TensorFlow (>=2.4) to use TFLite converter')

## 4. PyTorch Neural Network

### Prepare Data for PyTorch

In [ ]:
# Convert to PyTorch tensors
X_train_hd_s = X_train_scaled
X_test_hd_s = X_test_scaled
y_train_t = y_train
y_test_t = y_test

X_train_tensor = torch.FloatTensor(X_train_hd_s)
y_train_tensor = torch.FloatTensor(y_train_t).reshape(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_hd_s)
y_test_tensor = torch.FloatTensor(y_test_t).reshape(-1, 1)

print(f'PyTorch training tensor shape: {X_train_tensor.shape}')

### Build and Train PyTorch Model

In [ ]:
class PyTorchNN(nn.Module):
    def __init__(self, input_size):
        super(PyTorchNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model
model_t = PyTorchNN(input_size=20)
criterion = nn.BCELoss()
optimizer = optim.Adam(model_t.parameters(), lr=0.001)

# Training loop
model_t.train()
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model_t(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model_t.eval()
        with torch.no_grad():
            test_outputs = model_t(X_test_tensor)
            test_loss = criterion(test_outputs, y_test_tensor)
        model_t.train()
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

print('\nPyTorch training completed')

### Export PyTorch Model (TorchScript)

In [ ]:
# Export to TorchScript
model_t.eval()
example_input = torch.randn(1, 20)
traced_script_module = torch.jit.trace(model_t, example_input)
traced_script_module.save('pytorch_model.pt')
print('Saved PyTorch TorchScript model to pytorch_model.pt')

In [ ]:
# === ONNX export and onnxruntime example ===
# NEW CELL ADDED: ONNX export for cross-platform deployment
import onnx
import os

onnx_path = 'pytorch_model.onnx'

try:
    # Create an example input (CPU) matching the model input size
    example_input = torch.randn(1, X_train_hd_s.shape[1]).to('cpu')
    # Ensure model is on CPU for ONNX export
    model_t_cpu = model_t.to('cpu')
    model_t_cpu.eval()

    torch.onnx.export(
        model_t_cpu,
        example_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names = ['input'],
        output_names = ['output'],
        dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}}
    )
    print('Saved ONNX model to', onnx_path)

    # Quick onnxruntime inference check (if onnxruntime installed)
    try:
        import onnxruntime as ort
        sess = ort.InferenceSession(onnx_path)
        inp_name = sess.get_inputs()[0].name
        dummy_np = example_input.numpy().astype(np.float32)
        res = sess.run(None, {inp_name: dummy_np})
        print('ONNX runtime output shape:', res[0].shape)
    except Exception as e:
        print('onnxruntime not available or failed to run: ', e)
        print('To run ONNX inference locally: pip install onnxruntime')

except Exception as e:
    print('ONNX export failed:', e)
    print('Common fixes: move model to cpu and set torch.no_grad(), ensure example_input shape matches model input')

## 5. Model Evaluation

In [ ]:
# Evaluate Keras model
keras_predictions = (model_keras.predict(X_test_scaled, verbose=0) > 0.5).astype(int).flatten()
print('Keras Model Performance:')
print(f'Accuracy: {accuracy_score(y_test, keras_predictions):.4f}')
print(classification_report(y_test, keras_predictions))

# Evaluate PyTorch model
model_t.eval()
with torch.no_grad():
    pytorch_predictions = (model_t(X_test_tensor) > 0.5).int().numpy().flatten()
print('\nPyTorch Model Performance:')
print(f'Accuracy: {accuracy_score(y_test, pytorch_predictions):.4f}')
print(classification_report(y_test, pytorch_predictions))

## 6. Model Deployment

This section covers exporting models for production deployment.